### Домашнее задание Transformers Training (50 баллов)

В этом домашнем задании требуется обучить несколько Transformer-based моделей в задаче машинного перевода. Для обучения можно воспользоваться текущим проектом, так и реализовать свой пайплайн обучения. Если будете использовать проект, теги **TODO** проекта отмечают, какие компоненты надо реализовать.
В ноутбуке нужно только отобразить результаты обучения и выводы. Архитектура модели(количетсво слоев, размерность и тд) остается на ваш выбор.

Ваш код обучения нужно выложить на ваш github, в строке ниже дать ссылку на него. В первую очередь будут оцениваться результаты в ноутбуке, код нужен для проверки адекватности результатов. 

Обучать модели до конца не нужно, только для демонстрации, что модель обучается и рабочая - снижение val_loss, рост bleu_score.

#### Сcылка на ваш github с проектом(вставить свой) - https://github.com/runnerup96/pytorch-machine-translation

Ноутбук с результатами выкладывать на ваш **google диск** курса. 

### Данные

`
wget https://www.manythings.org/anki/rus-eng.zip && unzip rus-eng.zip
`

Модели нужно обучить на задаче перевода с английского на русский. 

### Обучение Seq2seq Transformer модель(25 баллов)

Реализуйте Seq2seq Transformer. В качестве блока трансформера можно использовать https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html. В качестве токенизатора воспользуйтесь HuggingFace токенизатор для source/target языков - https://huggingface.co/docs/transformers/fast_tokenizers
В качестве максимальной длинны возьмите предложения длинной **до 15 слов**, без каких либо префиксов. 

Не забудьте остальные элементы модели:
* Мы можем использовать 1 трансформер как энкодер - декодером будет выступать линейный слой. 
* Обучите свой BPE токенизатор - https://huggingface.co/docs/transformers/fast_tokenizers
* Матрицу эмбеддингов токенов
* Матрицу позицонных эмбеддингов
* Линейный слой проекции в target словарь
* Функцию маскирования будущих состояний attention, так как модель авто-регрессионна
* Learning rate schedualer


В качестве результатов, приложите слудующие данные:
1) Параметры обучения - learning rate, batch_size, epoch_num, размерность скрытого слоя, количетсво слоев
2) Графики обучения - train loss, val loss, bleu score
3) Примеры переводов вашей модели(10 штук) - source text, true target text, predicted target text

In [1]:
from src.data.utils import TextUtils, short_text_filter_function

def prepare_data(filepath, max_length=15):
    pairs = TextUtils.read_langs_pairs_from_file(filename=filepath)
    
    filtered_pairs = [pair for pair in pairs if short_text_filter_function(pair, max_length=max_length)]
    
    english_sentences = [pair[0] for pair in filtered_pairs]
    russian_sentences = [pair[1] for pair in filtered_pairs]

    return english_sentences, russian_sentences

english, russian = prepare_data('data/rus.txt')
print("Примеры предложений:")
for i in range(3):
    print(english[i], " --> ", russian[i])


Reading from file: 100%|██████████| 496059/496059 [00:06<00:00, 72247.80it/s]


Примеры предложений:
go  -->  марш
go  -->  иди
go  -->  идите


In [2]:
print("Примеры предложений:")
for i in range(-3, -1):
    print(english[i], " --> ", russian[i])

Примеры предложений:
there s a huge difference between making oneself understood and completely mastering english  -->  есть огромная разница между быть понятным другим и правильно говорить по английски
every student of biology anatomy anthropology ethnology or psychology is familiar with these facts  -->  каждый студент изучающий биологию анатомию антропологию этнологию и психологию знаком с этими фактами


In [1]:
from src.data.bpe_tokenizer import BPETokenizer
from src.data.datamodule import DataManager
from src.models.seq2seq_transformer import Seq2SeqTransformer
from src.models.trainer import Trainer
from src.txt_logger import TXTLogger
from tokenizers import Tokenizer
import torch

import warnings
warnings.simplefilter("ignore")

d:\Programs\Anaconda3\envs\ML311\Lib\site-packages\torchtext\data\__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


In [2]:
# source_tokenizer = Tokenizer.from_file("source_tokenizer.json")
# target_tokenizer = Tokenizer.from_file("target_tokenizer.json")

In [3]:
DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
data_manager = DataManager(
    config={
        "prefix_filter": [],
        "filename": 'data/rus.txt',
        "max_length": 15,
        "train_size": 0.8,
        "batch_size": 128
    },
    device=DEVICE
)
train_dataloader, val_dataloader = data_manager.prepare_data()

Reading from file: 100%|██████████| 496059/496059 [00:06<00:00, 73767.22it/s]


In [8]:
model = Seq2SeqTransformer(device=DEVICE,
                          embedding_size=256,
                          num_encoder_layers=8,
                          dim_feedforward=4096,
                          src_voc_size=len(data_manager.source_tokenizer.tokenizer.get_vocab()),
                          trg_voc_size=len(data_manager.target_tokenizer.tokenizer.get_vocab()),
                          target_tokenizer=data_manager.target_tokenizer,
                          source_tokenizer=data_manager.source_tokenizer,
                          lr_decay_step=5,
                          lr = 1e-4,
                          lr_decay=1e-1
                          )

In [9]:
trainer_config = {
    'epoch_num': 10
}

logger = TXTLogger(work_dir="./logs")
trainer = Trainer(model, trainer_config, logger)

In [10]:
trainer.train(train_dataloader, val_dataloader)

  0%|          | 0/10 [00:00<?, ?it/s]


RuntimeError: stack expects each tensor to be equal size, but got [7] at entry 0 and [5] at entry 1

In [ ]:
target_tokenizer.get_vocab_size()

30000

In [20]:
model_config = {
    'num_encoder_layers': 3,
    'num_decoder_layers': 3,
    'emb_size': 512,
    'nhead': 8,
    'vocab_size': max(source_tokenizer.get_vocab_size(), target_tokenizer.get_vocab_size()),
    'max_len': 50,
    'learning_rate': 1e-4
}
model = Seq2SeqTransformer(**model_config, device=data_manager.device)


TypeError: Seq2SeqTransformer.__init__() got an unexpected keyword argument 'device'

In [16]:
# from src.data.utils import TextUtils, short_text_filter_function

# filename = 'data/rus.txt'
# pairs = TextUtils.read_langs_pairs_from_file(filename)
# filtered_pairs = [pair for pair in pairs if short_text_filter_function(pair, 15)]

In [11]:
from src.models.trainer import Trainer
from src.models.seq2seq_transformer import Seq2SeqTransformer
from src.data.datamodule import DataManager
from src.data.bpe_tokenizer import BPETokenizer
import torch

In [10]:
# Загружаем токенизатор
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("trained_tokenizer.json")

# Функция для токенизации и детокенизации предложения
def test_tokenizer(tokenizer, sentence):
    encoded = tokenizer.encode(sentence)
    print("Токены:", encoded.ids)
    decoded = tokenizer.decode(encoded.ids)
    print("Декодированное предложение:", decoded)

# Тестируем на примере
test_sentence = "Hello, how are you?"
test_tokenizer(tokenizer, test_sentence)


Токены: [0, 9, 6314, 0, 257, 171, 70, 0]
Декодированное предложение: e llo how are you


In [12]:
config = {
    'epoch_num': 10,
    'batch_size': 32,
    'learning_rate': 0.001,
    'emb_size': 512,
    'nhead': 8,
    'num_encoder_layers': 3,
    'num_decoder_layers': 3,
    'dim_feedforward': 2048,
    'dropout': 0.1,
    'src_vocab_size': len(tokenizer.tokenizer.get_vocab()),
    'tgt_vocab_size': len(tokenizer.tokenizer.get_vocab()),
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'filename': 'data/rus.txt',
    'max_length': 15,
    'train_size': 0.8
}

AttributeError: 'tokenizers.Tokenizer' object has no attribute 'tokenizer'

In [23]:
from src.data.bpe_tokenizer import BPETokenizer

# Объединяем английские и русские предложения для обучения токенизатора
all_sentences = english + russian

# Создаем и обучаем BPE токенизатор
bpe_tokenizer = BPETokenizer(all_sentences)

# Сохраняем токенизатор для последующего использования
bpe_tokenizer.tokenizer.save("trained_tokenizer.json")


In [24]:
# Загружаем токенизатор
from tokenizers import Tokenizer

tokenizer = Tokenizer.from_file("trained_tokenizer.json")

# Функция для токенизации и детокенизации предложения
def test_tokenizer(tokenizer, sentence):
    encoded = tokenizer.encode(sentence)
    print("Токены:", encoded.ids)
    decoded = tokenizer.decode(encoded.ids)
    print("Декодированное предложение:", decoded)

# Тестируем на примере
test_sentence = "Hello, how are you?"
test_tokenizer(tokenizer, test_sentence)


Токены: [0, 9, 6314, 0, 257, 171, 70, 0]
Декодированное предложение: e llo how are you


In [25]:
import torch

In [27]:
config = {
    'epoch_num': 10,
    'batch_size': 32,
    'learning_rate': 0.001,
    'src_vocab_size': len(bpe_tokenizer.tokenizer.get_vocab()),
    'tgt_vocab_size': len(bpe_tokenizer.tokenizer.get_vocab()),
    'emb_size': 512,
    'nhead': 8,
    'num_encoder_layers': 3,
    'num_decoder_layers': 3,
    'dim_feedforward': 2048,
    'dropout': 0.1,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu'
}


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class TranslationDataset(Dataset):
    def __init__(self, src_data, tgt_data):
        self.src_data = src_data
        self.tgt_data = tgt_data

    def __len__(self):
        return len(self.src_data)

    def __getitem__(self, idx):
        src = torch.tensor(self.src_data[idx])
        tgt = torch.tensor(self.tgt_data[idx])
        return src, tgt

# Создание объектов Dataset
train_dataset = TranslationDataset(tokenized_src_train, tokenized_tgt_train)
val_dataset = TranslationDataset(tokenized_src_val, tokenized_tgt_val)


In [ ]:
from src.models.trainer import Trainer
from src.models.seq2seq_transformer import Seq2SeqTransformer

# Инициализация модели
model = Seq2SeqTransformer(
    num_encoder_layers=config['num_encoder_layers'],
    num_decoder_layers=config['num_decoder_layers'],
    emb_size=config['emb_size'],
    nhead=config['nhead'],
    src_vocab_size=config['src_vocab_size'],
    tgt_vocab_size=config['tgt_vocab_size'],
    dim_feedforward=config['dim_feedforward'],
    dropout=config['dropout']
).to(config['device'])

# Обучение модели
trainer = Trainer(model=model, model_config=config)
trainer.train(train_dataloader, val_dataloader)


### Fine-tune pretrained T5 (25 баллов)

Реализуйте Seq2seq Pretrained T5. Воспользуйтесь https://huggingface.co/docs/transformers/model_doc/t5 предобученной моделью. В качестве максимальной длинны возьмите предложения длинной **до 15 слов**, без каких либо префиксов. Архитектура модели(количетсво слоев, размерность и тд) остается на ваш выбор.

Не забудьте важные аспекты обучения модели:
* Взять готовый t5 токенизатор
* Resize matrix embedding - скорей всего ваша матрица эмбеддингов не будет включать эмбеддинги из вашего сета. Пример обновления матрицы эмбеддингов тут тут https://github.com/runnerup96/Transformers-Tuning/blob/main/t5_encoder_decoder.py
* Learning rate schedualer/Adafactor with constant learning rate


В качестве результатов, приложите слудующие данные:
1) Параметры обучения - learning rate, batch_size, epoch_num, pretrained model name
2) Графики обучения - train loss, val loss, bleu score
3) Примеры переводов вашей модели(10 штук) - source text, true target text, predicted target text

In [6]:
# CODE HERE